<a href="https://colab.research.google.com/github/wwwh0225/elements-of-statistical-learning/blob/main/Ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自由度與 Smoother Matrices

在先前的討論，我們尚未提到 smoothing spline 自由度的問題，若我們有預先決定好的(pre-chosen) $\lambda$ ，所求出的 $\mathbf{\hat{f}}$ 就會是 $\mathbf{y}$ 的一種線性組合，故稱 linear smoother 。

$$\begin{aligned}
\mathbf{\hat{f}} &= \mathbf{N}(\mathbf{N}^T\mathbf{N}+\lambda\mathbf{\Omega_N})^{-1}\mathbf{N}^T\mathbf{y} \\
&=  \mathbf{S_\lambda y}
\end{aligned}$$

我們稱 $\mathbf{S_\lambda}$ 為 **smoother matrix**。

(顯而易見地， $\mathbf{S_\lambda}$ 與 $x_i$ 和 $\lambda$ 有關 )

而我們再來看到 cubic spline 對函數 $f$ 的估計，我們令 $\mathbf{B_\xi}$ 是一個 $X \times M$ 的矩陣，表示 M 個 cubic-spline basis functions，且一共有 N 個訓練點，並以 $\xi$ 表達 knots 的序列，以及 $M\ll N$ (確保不會有奇異矩陣)。


![](https://latex.codecogs.com/gif.latex?%5Cbegin%7Baligned%7D%20%5Cmathbf%7B%5Chat%7Bf%7D%7D%20%26%3D%20%5Cmathbf%7BB_%5Cxi%28B_%5Cxi%5ETB_%5Cxi%29%5E%7B-1%7DB_%5Cxi%5ETy%7D%20%5C%5C%20%26%3D%20%5Cmathbf%7BH_%5Cxi%20y%7D%20%5Cend%7Baligned%7D)


從上述結果可以看出  $\mathbf{H_\xi}$ 是個投影矩陣，具備以下性質

- 對稱、半正定
- 自乘不變，也就是 $\mathbf{H_\xi}\mathbf{H_\xi}=\mathbf{H_\xi}$

(另外 $rank(\mathbf{H_\xi})=M$ 、  $rank(\mathbf{S_\lambda})=N$  )

$\mathbf{H_\xi}$ 提供了維度，也讓我們知道會需要幾個 basis function 。

我們說 smoothing spline 的 effective degrees of freedom 為：

$$df_\lambda = trace(\mathbf{S_\lambda})$$

由於 $\mathbf{S_\lambda}$ 是對稱切半正定矩陣，故其擁有時數的特徵值，我們可以對它進行特徵分界 (Eigendecomposition)，而為了較好地分解，我們可將 $\mathbf{S_\lambda}$ 先改寫成所謂 **Reinsch form** 。


### Reinsch form 的推導 (from Ex. 5.9)

**Derive the Reinsch form $\mathbf{S_\lambda} = (\mathbf{I} + \lambda \mathbf{K})^{-1}$ for the smoothing spline.**

$\mathbf{N}$ 是一個 $N \times N$ 的方陣，對其做奇異值分解 $\mathbf{N} = UDV^T$ ， $U$ 是正交矩陣 $U^{-1}=U^T$ 且因為 $\mathbf{N}$ 是 full-rank ，故 $D$ 也可以取反矩陣，因此

$$\begin{aligned}
\mathbf{S_\lambda} &= UDV^T(VD^2V^T+\lambda\mathbf{\Omega_N})^{-1}VDU^T \\
&=U(D^{-1}V^TVD^2V^TVD^{-1}+\lambda D^{-1}V^T\mathbf{\Omega_N}VD^{-1})^{-1}U^T \\
&= U(I+\lambda D^{-1}V^T \mathbf{\Omega_N} VD^{-1})^{-1}U^T\\
&= (U^TU+\lambda U^TD^{-1}V^T\mathbf{\Omega_N}VD^{-1}U)^{-1}\\
&= (I+\lambda \underbrace{  U^TD^{-1}V^T\mathbf{\Omega_N}VD^{-1}U }_{\mathbf{K}})^{-1} \\
&= (I+\lambda \mathbf{K})^{-1}
\end{aligned} $$


---

根據上面所求得的結果， 可以發現 $\mathbf{K}$ 和 $\lambda$ 沒有關係。而我們之前所說的 $\hat{ \mathbf{f}} = \mathbf{S_\lambda y}$ 就是下面這條式子的解：

$$\min_{\mathbf{f}} (\mathbf{y-f})^T (\mathbf{y-f})+ \lambda \mathbf{f}^T \mathbf{K} \mathbf{f}$$

其中 $\mathbf{K}$ 是所謂**懲罰矩陣**，而我們再對 $\mathbf{S_\lambda y}$ 做特徵值分解，可以得到下列形式：

$$\mathbf{S_\lambda y} = \sum_{k=1}^N \rho_k(\lambda)\mathbf{u}_k\mathbf{u}_k^T $$

$$\rho_k(\lambda)= \frac{1}{1+\lambda d_k}$$

其中， $d_k$ 是 $\mathbf{K}$ 對應的特徵值、$\mathbf{u}_k$ 則是對應的特徵向量。

(其餘特徵值相關內容可見課本 p154 ~ p156)

### $\lambda$ 的選擇
延伸閱讀：https://esl.hohoweiya.xyz/notes/spline/sim-5-9/index.html

在進行 smoothing spline 的配適時，我們還要去找到一個最適的 $\lambda$ 我們希望可以找到一個 $\lambda$ 使得 cross-validated RSS 之值最小：(LOO(leave-one-out CV a.k.a n-fold CV))

$$RSS_{cv}(\lambda) = \sum_{i=1}^n(y_i-\hat{f}_\lambda^{(-i)}(x_i))^2= \sum_{i=1}^n [\frac{y_i-\hat{f}_\lambda(x_i)}{1-\{\mathbf{S_\lambda}\}_{ii}}]^2$$

其中 $\hat{f}_\lambda^{(-i)}(x_i)$ 為移除第 $i$ 個樣本點(of training set)後執行 $LOO$ 代入 $x_i$ 配適值。

除了上方傳統的 LOOCV 之外，我們還可以引入 Generalized CV (GCV) 同樣也可以用以找尋最佳的 $\lambda$ ，GCV 的好處是在計算上較LOOCV更加便利及快速。

$$GCV =  \sum_{i=1}^n [\frac{y_i-\hat{f}_\lambda(x_i)}{1-\frac{1}{n}trace(\mathbf{S_\lambda})}]^2$$
 
## Nonparametric Logistic Regression